# <font color='blue'>Data Science Academy</font>
# <font color='blue'>Deep Learning I</font>

## Construindo Um Algoritmo Para Rede Neural Multilayer Perceptron

### Função de Custo ou Perda

Neste ponto, você usou pesos e bias para calcular saídas. E você usou uma função de ativação para categorizar a saída. Como você pode se lembrar, as redes neurais melhoram a precisão de suas saídas, modificando pesos e bias em resposta ao treinamento contra conjuntos de dados rotulados (aprendizagem supervisionada).

Existem muitas técnicas para definir a precisão de uma rede neural, que se centra na capacidade da rede de produzir valores que se aproximem dos valores corretos conhecidos. As pessoas usam nomes diferentes para essa medida de precisão, muitas vezes significando perda ou custo. Usaremos o termo custo mais frequentemente.

Vamos calcular o custo usando o erro quadrático médio (MSE). Essa é a fórmula do MSE:

In [1]:
from IPython.display import Image
Image(url = '15-MLP05-01.png')

Aqui w indica a coleta de todos os pesos na rede, b todos os bias, m é o número total de exemplos (instâncias) de treinamento, a é a aproximação de y(x) pela rede e tanto a como y(x) são vetores de mesmo comprimento.

A coleção de pesos é o conjunto de todas as matrizes de peso "achatadas" (flattened) em vetores e concatenadas em um grande vetor. O mesmo vale para a coleta de bias, exceto que eles já são vetores, então não há necessidade de "achatá-los" (flatten) antes da concatenação. Veja um exemplo desta operação:

In [2]:
import numpy as np

# Matrizes 2x2
w1  = np.array([[1, 2], [3, 4]])
w2  = np.array([[5, 6], [7, 8]])

# flatten
w1_flat = np.reshape(w1, -1)
w2_flat = np.reshape(w2, -1)

w = np.concatenate((w1_flat, w2_flat))
print(w)

[1 2 3 4 5 6 7 8]


É uma ótima maneira de abstrair todos os pesos e bias usados ​​na rede neural e torna algumas tarefas mais fáceis, como veremos daqui a pouco.

O custo C (da fórmula acima), depende da diferença entre a saída correta, y(x) e a saída da rede, a. É fácil ver que nenhuma diferença entre y(x) e a (para todos os valores de x) leva a um custo de 0. Esta é a situação ideal e, de fato, o processo de aprendizagem gira em torno de minimizar o custo, tanto quanto possível.

Vamos então calcular o custo!

Apenas una observação: a ativação do nó sigmóide não significa nada porque a rede não possui saída rotulada para comparar. Além disso, os pesos e o bias não podem mudar e o aprendizado não pode acontecer sem um custo.

In [2]:
import numpy as np


class Neuronio(object):
    """
    Classe base para os nós na rede.

    Argumentos:

        `nodes_entrada`: Uma lista de nós com arestas para este nó.
    """
    def __init__(self, nodes_entrada = []):
        """
        O construtor do nó (é executado quando o objeto é instanciado). 
        Define propriedades que podem ser usadas por todos os nós.
        """
        self.nodes_entrada = nodes_entrada
        self.nodes_saida = []
        self.valor = None
        for n in nodes_entrada:
            n.nodes_saida.append(self)

    def forward(self):
        raise NotImplementedError


class Input(Neuronio):
    """
    Uma entrada genérica na rede.
    """
    def __init__(self):
        Neuronio.__init__(self)

    def forward(self):
        pass


class Linear(Neuronio):
    """
    Representa um nó que executa uma transformação linear.
    """
    def __init__(self, X, W, b):
        # O construtor da classe base (nó). Pesos e bias são tratados como nós de entrada.
        Neuronio.__init__(self, [X, W, b])

    def forward(self):
        """
        Executa a matemática por trás de uma transformação linear.
        """
        X = self.nodes_entrada[0].valor
        W = self.nodes_entrada[1].valor
        b = self.nodes_entrada[2].valor
        self.valor = np.dot(X, W) + b


class Sigmoid(Neuronio):
    """
    Representa um nó que executa a função de ativação sigmoid.
    """
    def __init__(self, node):
        Neuronio.__init__(self, [node])

    def _sigmoid(self, x):
        """
        Este método é separado do `forward` porque ele também será usado com "backward".

        `x`: numpy array object.
        """
        return 1. / (1. + np.exp(-x))

    def forward(self):
        """
        Executa a função sigmoid e define o valor.
        """
        input_value = self.nodes_entrada[0].valor
        self.valor = self._sigmoid(input_value)


class CostFunction(Neuronio):
    def __init__(self, y, a):
        """
        Função do custo médio do erro. Deve ser usado como o último nó para uma rede.
        """
        Neuronio.__init__(self, [y, a])

    def forward(self):
        """
        Calcula o erro quadrático médio.
        """
        # NOTE: Aplicamos a função reshape() para evitar possíveis erros nas operações de matriz / vetor,
        # conforme vimos nas aulas de álgebra linear.
        #
        # Tornando ambos os arrays(3,1) asseguramos que o resultado seja (3,1).
        y = self.nodes_entrada[0].valor.reshape(-1, 1)
        a = self.nodes_entrada[1].valor.reshape(-1, 1)
        m = self.nodes_entrada[0].valor.shape[0]

        diff = y - a
        self.valor = np.mean(diff**2)


def topological_sort(feed_dict):
    """
    Classifica os nós em ordem topológica usando o Algoritmo de Kahn.

    `Feed_dict`: um dicionário em que a chave é um nó ` Input` e o valor é o respectivo feed de valor para esse nó.

    Retorna uma lista de nós ordenados.
    """

    input_nodes = [n for n in feed_dict.keys()]

    G = {}
    nodes = [n for n in input_nodes]
    while len(nodes) > 0:
        n = nodes.pop(0)
        if n not in G:
            G[n] = {'in': set(), 'out': set()}
        for m in n.nodes_saida:
            if m not in G:
                G[m] = {'in': set(), 'out': set()}
            G[n]['out'].add(m)
            G[m]['in'].add(n)
            nodes.append(m)

    L = []
    S = set(input_nodes)
    while len(S) > 0:
        n = S.pop()

        if isinstance(n, Input):
            n.valor = feed_dict[n]

        L.append(n)
        for m in n.nodes_saida:
            G[n]['out'].remove(m)
            G[m]['in'].remove(n)
            if len(G[m]['in']) == 0:
                S.add(m)
    return L


def forward_pass(output_node, sorted_nodes):
    """
   Executa uma passagem para a frente através de uma lista de nós ordenados.

     Argumentos:

         `Output_node`: Um nó no grafo, deve ser o nó de saída.
         `Sorted_nodes`: uma lista topologicamente ordenada de nós.

     Retorna o valor do Nó de saída
    """

    for n in sorted_nodes:
        n.forward()

    return output_node.valor

A matemática por trás do MSE reflete a equação no início deste Jupyter Notebook, onde y é saída esperada e a é gerada pela rede neural. Nós então calculamos esta diferença e elevamos o resultado ao quadrado. Por último, precisamos somar as diferenças ao quadrado e dividir pelo número total de exemplos m. Isso pode ser alcançado com np.mean ou (1 / m) * np.sum (diff ** 2).

Observe que a ordem de y e a realmente não importa, nós poderíamos alterá-los (a - y) e obter o mesmo valor.

### Executando o Grafo

In [3]:
# Parâmetros de entrada
inputs, weights, bias = Input(), Input(), Input()
y = Input()

# Função Linear
f = Linear(inputs, weights, bias)
g = Sigmoid(f)

# Função de Custo
cost = CostFunction(y, g)

# Atribuindo valores aos parâmetros
x = np.array([[-2., -1.], [-2, -4]])
w = np.array([[4., -6], [3., -2]])
b = np.array([-2., -3])

# Valores de saída (observados)
array_y = np.array([[-4., -2.], [-1, -3]])

# Define o feed_dict
feed_dict = {inputs: x, weights: w, bias: b, y: array_y}

# Ordena as entradas para execução
graph = topological_sort(feed_dict)

# Gera o output com o forward_pass. Perceba que usamos o objeto cost aqui!!!!!
output = forward_pass(cost, graph)

# Print
print(cost.valor)

10.499979385927558


## Fim